<a href="https://colab.research.google.com/github/EAkeweje/Accelerating-Hydrogen-Oxidation-Calculations/blob/main/FNNs_ModelOpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -Uq

     |████████████████████████████████| 1.8 MB 15.9 MB/s 
     |████████████████████████████████| 181 kB 63.2 MB/s 
     |████████████████████████████████| 162 kB 64.0 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 158 kB 70.1 MB/s 
     |████████████████████████████████| 157 kB 73.3 MB/s 
     |████████████████████████████████| 157 kB 71.3 MB/s 
     |████████████████████████████████| 157 kB 58.5 MB/s 
     |████████████████████████████████| 157 kB 77.4 MB/s 
     |████████████████████████████████| 157 kB 76.8 MB/s 
     |████████████████████████████████| 157 kB 77.0 MB/s 
     |████████████████████████████████| 157 kB 66.4 MB/s 
     |████████████████████████████████| 156 kB 76.1 MB/s 


In [ ]:
#import and login
import wandb

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json

import torch
from sklearn.metrics import r2_score
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from torchsummary import summary

import os
import sys

## Utils and network

In [ ]:
#get utils

sys.path.append(os.getcwd()+'/scripts')
import Wandb_Utils as wu

In [ ]:
class net(nn.Module):
  def __init__(self, config):
    super(net, self).__init__()
    self.in_dim, self.out_dim = config.input_size, config.output_size
    self.nhid = config.num_layers
    self.hid_dims = config.hidden_size

    self.in_layer = nn.Linear(self.in_dim, self.hid_dims[0])
    current_dim = self.hid_dims[0]
    self.hid_layers = nn.ModuleList()
    for i in range(self.nhid-1):
      self.hid_layers.append(nn.Linear(current_dim, self.hid_dims[1]))
      current_dim = self.hid_dims[1]
    self.hid_layers.append(nn.Linear(current_dim, self.hid_dims[0]))
    self.out_layer = nn.Linear(self.hid_dims[0], self.out_dim)
    self.relu = nn.ReLU()
    self.in_batchnorm = nn.BatchNorm1d(self.hid_dims[0])
    self.hid_batchnorm = nn.BatchNorm1d(self.hid_dims[1])
    self.out_batchnorm = nn.BatchNorm1d(self.hid_dims[0])
    self.dropout = nn.Dropout(p = config.dropout)

  def forward(self, x):
    #pass input to input layer
    out = self.in_batchnorm(self.relu(self.in_layer(x)))
    #pass through hidden layers
    for i in range(self.nhid-1):
      out = self.hid_batchnorm(self.relu(self.hid_layers[i](self.dropout(out))))
    out = self.out_batchnorm(self.relu(self.hid_layers[-1](self.dropout(out))))
    #pass to output layer
    out = self.out_layer(self.dropout(out))

    return out
    
def init_weights(m):
  if type(m) == nn.Linear:
    nn.init.kaiming_uniform_(m.weight, nonlinearity= 'relu')
    m.bias.data.fill_(0.0)

# Model Optimization

In [ ]:
#Define the sweep
sweep_config = {
    'name': 'Accelerating Hydrogen Oxidation Calculations (Concentration inputs, transformed data, Exclude pressure, Exclude H2 and O2 from model target, last steps)',
    'description': 'Model Optimization for Adam Optimizer',
    'method': 'bayes', #how sweep controller select hyperparameter
    'metric': {'name': 'loss',
               'goal': 'minimize'}
                }
param_dict = {
    'batch_size': {
        'values': [8, 16, 32, 64, 128]
        },
    'timesteps': {
        'value': [-1]
        },
    'nsample': {
        'value': 1500
        },
    'hidden_size': {
        'values': [[32, 128], [32, 256], [64, 256], [64, 512]]
        },
    'num_layers': {
        'values': [2, 3, 4]
        },
    'dropout': {
        'values': [0, 0.05, 0.1, 0.15, 0.2]
    },
    'epochs': {
        'value': 2500
        },
    'optimizer': {
        'value': 'Adam'
        },
    'lr': {
      'values': [5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4]
        },
    'weight_decay': {
        'values' : [0.05, 0.01, 0]
    },
    'beta1': {
        'values': [0.85, 0.9, 0.95]
        },
    'beta2': {
        'values': [0.9, 0.95, 0.99]
        },
    'mb_coeff': {
        'value': 0
        },
    'scheduling': {
        'values': [True, False]
       },
    'model': {
        'value': 'FNN'
    },
    'split': {
        'value': [0.7, 0.2, 0.1]
    },
    'dict_path':{
        'value': None
    },
    'mean_std_path': {
        'value': "ConcInputs_NoH2O2(taraget_only)_NoPressure_mean_std_laststep.json"
    },
    'input_size':{
        'value': 9
    },
    'output_size':{
        'value': 7
    },
    'in_conc': {
        'value': True
    },
    'inputs_path': {
        'value' : 'input_98660.npy'
    },
    'outputs_dir': {
        'value': './Out_files_npy'
    },
    'in_h2_o2': {
        'value': True
    },
    'out_h2_o2': {
        'value': False
    },
    'in_pres': {
        'value': False
    },
    'return_1d_input': {
        'value' : True
    }
    }
sweep_config['parameters'] = param_dict
sweep_config

{'name': 'Accelerating Hydrogen Oxidation Calculations (Concentration inputs, transformed data, Exclude pressure, Exclude H2 and O2 from model target, last steps)',
 'description': 'Model Optimization for Adam Optimizer',
 'method': 'bayes',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'batch_size': {'values': [8, 16, 32, 64, 128]},
  'timesteps': {'value': [-1]},
  'nsample': {'value': 1500},
  'hidden_size': {'values': [[32, 128], [32, 256], [64, 256], [64, 512]]},
  'num_layers': {'values': [2, 3, 4]},
  'dropout': {'values': [0, 0.05, 0.1, 0.15, 0.2]},
  'epochs': {'value': 2500},
  'optimizer': {'value': 'Adam'},
  'lr': {'values': [0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]},
  'weight_decay': {'values': [0.05, 0.01, 0]},
  'beta1': {'values': [0.85, 0.9, 0.95]},
  'beta2': {'values': [0.9, 0.95, 0.99]},
  'mb_coeff': {'value': 0},
  'scheduling': {'values': [True, False]},
  'model': {'value': 'FNN'},
  'split': {'value': [0.7, 0.2, 0.1]},
  'dict_path': {'val

In [ ]:
# Initialize Sweep
sweep_id = wandb.sweep(sweep_config, project="Hydrogen-Oxidation-Models-Optimization")

Create sweep with ID: iwmnhlen
Sweep URL: https://wandb.ai/data-bigwig/Hydrogen-Oxidation-Models-Optimization/sweeps/iwmnhlen


In [ ]:
# sweep_id = 'yxhoindr'

In [ ]:
def train_model(config = None):
  # tell wandb to get started
  with wandb.init(config=config):
  # access all HPs through wandb.config, so logging matches execution!
    config = wandb.config

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #Get data
    train_loader, val_loader, test_loader = wu.make_dataloaders(config)
    #obtaining mean and std of training set
    train_x_mean, train_x_std, train_y_mean, train_y_std = wu.load_mean_std(config.mean_std_path, train_loader)
    #initialize model
    model = net(config)

    #training
    model = model.to(device)
    mean_std = [train_x_mean.to(device), train_x_std.to(device), train_y_mean.to(device), train_y_std.to(device)]
    criterion  = nn.MSELoss()
    epochs, val_loss = wu.training(model, train_loader, val_loader, config, criterion, mean_std, device)
    wandb.log({'loss': val_loss})
    wandb.log({'num_epochs': epochs})

In [ ]:
wandb.agent(sweep_id, train_model, project="Hydrogen-Oxidation-Models-Optimization", count=20)

wandb: Agent Starting Run: oqkc9w5c with config:
wandb: 	batch_size: 128
wandb: 	beta1: 0.9
wandb: 	beta2: 0.99
wandb: 	dict_path: None
wandb: 	dropout: 0.05
wandb: 	epochs: 2500
wandb: 	hidden_size: [64, 256]
wandb: 	in_conc: True
wandb: 	in_h2_o2: True
wandb: 	in_pres: False
wandb: 	input_size: 9
wandb: 	inputs_path: input_98660.npy
wandb: 	lr: 0.01
wandb: 	mb_coeff: 0
wandb: 	mean_std_path: ConcInputs_NoH2O2(taraget_only)_NoPressure_mean_std_laststep.json
wandb: 	model: FNN
wandb: 	nsample: 1500
wandb: 	num_layers: 4
wandb: 	optimizer: Adam
wandb: 	out_h2_o2: False
wandb: 	output_size: 7
wandb: 	outputs_dir: ./Out_files_npy
wandb: 	return_1d_input: True
wandb: 	scheduling: False
wandb: 	split: [0.7, 0.2, 0.1]
wandb: 	timesteps: [-1]
wandb: 	weight_decay: 0
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: data-bigwig. Use `wandb login --relogi

learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁
num_epochs,▁
train_loss,█▅▃▃▃▃▃▃▂▃▃▂▄▃▂▂▂▃▂▃▁▂▄▁▂▄▂▂▂▂▂▁▂▂▂▂▁▃▂▂
val_loss,█▃▃▃▄▄▄▁▄▃▄▃▅▄▃▅▄▅▅▅▅▄▅▅▆▆▆▆▇▇█▆▇▇▅██▆▅▆
learning_rate,0.01
loss,0.34754
num_epochs,245
train_loss,0.03889
val_loss,0.21709


wandb: Agent Starting Run: 8y6k78uh with config:
wandb: 	batch_size: 64
wandb: 	beta1: 0.95
wandb: 	beta2: 0.9
wandb: 	dict_path: None
wandb: 	dropout: 0.05
wandb: 	epochs: 2500
wandb: 	hidden_size: [32, 256]
wandb: 	in_conc: True
wandb: 	in_h2_o2: True
wandb: 	in_pres: False
wandb: 	input_size: 9
wandb: 	inputs_path: input_98660.npy
wandb: 	lr: 0.05
wandb: 	mb_coeff: 0
wandb: 	mean_std_path: ConcInputs_NoH2O2(taraget_only)_NoPressure_mean_std_laststep.json
wandb: 	model: FNN
wandb: 	nsample: 1500
wandb: 	num_layers: 2
wandb: 	optimizer: Adam
wandb: 	out_h2_o2: False
wandb: 	output_size: 7
wandb: 	outputs_dir: ./Out_files_npy
wandb: 	return_1d_input: True
wandb: 	scheduling: False
wandb: 	split: [0.7, 0.2, 0.1]
wandb: 	timesteps: [-1]
wandb: 	weight_decay: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁
num_epochs,▁
train_loss,▆▄▆▃▆▆▇█▄▃▂▄▅▃▄▃▃▄▁▂▁▃▂▃▄▄▃▁▄▄▂▂▃▅▂▂▂▃▁▃
val_loss,▅▃▇▇█▂▆█▂▁▃▄▃▁▅▂▂▇▁▄▂▅▃▂▃▃▁▂▂▄▁▂▃▄▁▁▁▂▃▁
learning_rate,0.05
loss,0.04383
num_epochs,548
train_loss,0.05742
val_loss,0.02081


wandb: Agent Starting Run: d4sjhb41 with config:
wandb: 	batch_size: 128
wandb: 	beta1: 0.85
wandb: 	beta2: 0.9
wandb: 	dict_path: None
wandb: 	dropout: 0.2
wandb: 	epochs: 2500
wandb: 	hidden_size: [32, 256]
wandb: 	in_conc: True
wandb: 	in_h2_o2: True
wandb: 	in_pres: False
wandb: 	input_size: 9
wandb: 	inputs_path: input_98660.npy
wandb: 	lr: 0.01
wandb: 	mb_coeff: 0
wandb: 	mean_std_path: ConcInputs_NoH2O2(taraget_only)_NoPressure_mean_std_laststep.json
wandb: 	model: FNN
wandb: 	nsample: 1500
wandb: 	num_layers: 2
wandb: 	optimizer: Adam
wandb: 	out_h2_o2: False
wandb: 	output_size: 7
wandb: 	outputs_dir: ./Out_files_npy
wandb: 	return_1d_input: True
wandb: 	scheduling: False
wandb: 	split: [0.7, 0.2, 0.1]
wandb: 	timesteps: [-1]
wandb: 	weight_decay: 0.01


...

learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁
num_epochs,▁
train_loss,█▃▂▂▂▂▂▁▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,0.0001
loss,0.02307
num_epochs,482
train_loss,0.02244
val_loss,0.01158


wandb: Agent Starting Run: zupm8o7y with config:
wandb: 	batch_size: 128
wandb: 	beta1: 0.95
wandb: 	beta2: 0.95
wandb: 	dict_path: None
wandb: 	dropout: 0.05
wandb: 	epochs: 2500
wandb: 	hidden_size: [64, 512]
wandb: 	in_conc: True
wandb: 	in_h2_o2: True
wandb: 	in_pres: False
wandb: 	input_size: 9
wandb: 	inputs_path: input_98660.npy
wandb: 	lr: 0.0001
wandb: 	mb_coeff: 0
wandb: 	mean_std_path: ConcInputs_NoH2O2(taraget_only)_NoPressure_mean_std_laststep.json
wandb: 	model: FNN
wandb: 	nsample: 1500
wandb: 	num_layers: 2
wandb: 	optimizer: Adam
wandb: 	out_h2_o2: False
wandb: 	output_size: 7
wandb: 	outputs_dir: ./Out_files_npy
wandb: 	return_1d_input: True
wandb: 	scheduling: True
wandb: 	split: [0.7, 0.2, 0.1]
wandb: 	timesteps: [-1]
wandb: 	weight_decay: 0


learning_rate,████▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁
num_epochs,▁
train_loss,█▃▂▂▂▂▂▂▂▁▁▂▂▁▁▁▂▁▂▂▂▂▁▁▁▂▁▁▂▁▁▁▃▁▁▁▁▁▁▁
val_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,0.0
loss,0.02162
num_epochs,1006
train_loss,0.0493
val_loss,0.00984


wandb: Agent Starting Run: 5krk32nn with config:
wandb: 	batch_size: 128
wandb: 	beta1: 0.9
wandb: 	beta2: 0.99
wandb: 	dict_path: None
wandb: 	dropout: 0
wandb: 	epochs: 2500
wandb: 	hidden_size: [32, 128]
wandb: 	in_conc: True
wandb: 	in_h2_o2: True
wandb: 	in_pres: False
wandb: 	input_size: 9
wandb: 	inputs_path: input_98660.npy
wandb: 	lr: 0.005
wandb: 	mb_coeff: 0
wandb: 	mean_std_path: ConcInputs_NoH2O2(taraget_only)_NoPressure_mean_std_laststep.json
wandb: 	model: FNN
wandb: 	nsample: 1500
wandb: 	num_layers: 2
wandb: 	optimizer: Adam
wandb: 	out_h2_o2: False
wandb: 	output_size: 7
wandb: 	outputs_dir: ./Out_files_npy
wandb: 	return_1d_input: True
wandb: 	scheduling: True
wandb: 	split: [0.7, 0.2, 0.1]
wandb: 	timesteps: [-1]
wandb: 	weight_decay: 0


learning_rate,█████▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁
num_epochs,▁
train_loss,█▆▆▆▆▆▃▂▃▃▂▃▂▁▃▄▄▃▂▁▁▁▄▃▃▂▃▃▂▂▃▂▂▂▂▁▁▂▂▂
val_loss,█▆▂▄▃▂▂▂▂▁▁▂▁▁▁▂▁▁▂▁▁▁▁▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁
learning_rate,1e-05
loss,0.00721
num_epochs,901
train_loss,0.01262
val_loss,0.00354
